In [2]:
import librosa
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import Wav2Vec2ForSequenceClassification, TrainingArguments, Trainer
import evaluate
from datasets import Dataset

c:\Users\DANIEL\anaconda3\envs\pfas\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.


In [3]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
# Assuming you have a DataFrame with columns "filename" and "emotion"
# data = pd.read_csv("C:/MyDocs/DTU/MSc/Thesis/Data/MELD/MELD_preprocess_test/pre_process_test.csv")
data = pd.read_csv("C:/Users/DANIEL/Desktop/thesis/low-resource-emotion-recognition/MELD_preprocess_test/pre_process_test.csv")

# directory = "C:/MyDocs/DTU/MSc/Thesis/Data/MELD/MELD_preprocess_test/MELD_preprocess_test_data"
directory = "C:/Users/DANIEL/Desktop/thesis/low-resource-emotion-recognition/MELD_preprocess_test/MELD_preprocess_test_data"

files = []

# Get a list of all files in the directory
for file in os.listdir(directory):
    if file.endswith('.wav'):
        files.append(file)

# Add filenames to a new column in the DataFrame
data['filename'] = files

features = []
labels = []

label_encoder = LabelEncoder()

raw_labels = data['Emotion'].values
labels = label_encoder.fit_transform(raw_labels)

# Show the label-encoding pairs:
print(label_encoder.classes_)
print("[0,         1,       2,       3,         4,         5,       6]")

print(labels)

max_length = 16000 * 6  # 10 seconds

for index, row in data.iterrows():

    # Load audio file
    file_to_load = row['filename']
    file_to_load_path = os.path.join(directory, file_to_load)
    # print()
    # print(index)
    # print(file_to_load)
    # print()

    audio, sr = librosa.load(file_to_load_path, sr=16000)
    
    if len(audio) > max_length:
        audio = audio[:max_length]
    else:
        padding = max_length - len(audio)
        offset = padding // 2
        audio = np.pad(audio, (offset, padding - offset), 'constant')
    
    
    features.append(audio)
    
    # Encode label
    # labels.append(label_encoder.transform([row['Emotion']]))

# Convert to arrays
features = np.array(features)
labels = np.array(labels).flatten()


# Now, `features` and `labels` can be used for training your model
# Optionally, save them to disk
# np.save('features.npy', features)
# np.save('labels.npy', labels)

print(features.shape)
print(labels.shape)

# Convert features to a float tensor and transpose the last two dimensions
features_tensor = torch.tensor(features).float()
labels_tensor = torch.tensor(labels).long()  # Use .long() for integer labels, .float() for one-hot

# Choose train indices and validation indices
train_indices = np.random.choice(len(features), int(0.8 * len(features)), replace=False)
val_indices = np.array([i for i in range(len(features)) if i not in train_indices])


# Convert the TensorDatasets to Datasets
train_dataset = Dataset.from_dict({
    'input_values': features_tensor[train_indices],
    'labels': labels_tensor[train_indices]
})
val_dataset = Dataset.from_dict({
    'input_values': features_tensor[val_indices],
    'labels': labels_tensor[val_indices]
})

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Load a pre-trained model for pretrained
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-large-xlsr-53", num_labels=7)

# Define training arguments
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

# Initialize the trainer
metric = evaluate.load("accuracy")

print(model.config)


['anger' 'disgust' 'joy' 'neutral' 'sadness' 'surprise']
[0,         1,       2,       3,         4,         5]
[4 5 3 2 4 3 3 2 3 5 3 3 5 0 3 2 3 3 3 5 5 5 2 3 3 2 5 3 3 5 3 3 3 0 3 3 5
 3 5 5 5 3 0 5 0 0 0 0 0 0 0 0 0 0 0 2 3 3 3 5 4 3 3 5 5 2 2 3 3 5 2 5 0 1
 2 2 3 3 3 3 3 3 3 2 3 3 3 3 3 3 2 3 4 3 2 3 3 3 3 3]
(100, 160000)
(100,)


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-xlsr-53",
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_d

In [6]:
# Prepare the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()



# Save the model
torch.save(model.state_dict(), 'emotion_recognition_model.pth')

c:\Users\DANIEL\anaconda3\envs\pfas\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
                                                 
 33%|███▎      | 10/30 [23:39<44:40, 134.00s/it]

{'eval_loss': 1.7677257061004639, 'eval_accuracy': 0.5, 'eval_runtime': 55.3417, 'eval_samples_per_second': 0.361, 'eval_steps_per_second': 0.054, 'epoch': 1.0}


                                                
 67%|██████▋   | 20/30 [48:06<25:00, 150.02s/it]

{'eval_loss': 1.7516752481460571, 'eval_accuracy': 0.5, 'eval_runtime': 61.4393, 'eval_samples_per_second': 0.326, 'eval_steps_per_second': 0.049, 'epoch': 2.0}


                                                  
100%|██████████| 30/30 [1:16:31<00:00, 153.03s/it]


{'eval_loss': 1.7456499338150024, 'eval_accuracy': 0.5, 'eval_runtime': 55.8138, 'eval_samples_per_second': 0.358, 'eval_steps_per_second': 0.054, 'epoch': 3.0}
{'train_runtime': 4591.0244, 'train_samples_per_second': 0.052, 'train_steps_per_second': 0.007, 'train_loss': 1.7601076761881511, 'epoch': 3.0}
